# Part 17

# Using Functions

Universal code for the entire notebook

In [ ]:
# Uncomment the line below to make sure you have all the packages needed
# %pip install -r requirements.txt

In [ ]:
# Standard library imports

# Third-party library imports
from openai import OpenAI  # Used for interacting with OpenAI's API
from openai import AssistantEventHandler  # Used for handling events related to OpenAI assistants
from typing_extensions import override  # Used for overriding methods in subclasses
from IPython.display import display, Markdown, clear_output  # Used for displaying content in Jupyter Notebooks


In [ ]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

## Simple Function Calling Example with Streaming

Note the changes to the event handler to deal with calling our functions (on_event, handle_requires_action, and submit_tool_outputs)

In [ ]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize an empty list to store the results

    @override
    def on_event(self, event):
        # Retrieve events that are denoted with 'requires_action'
        # since these will have our tool_calls
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)
    
    def handle_requires_action(self, data, run_id):
        tool_outputs = []
        
        
        
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "get_current_temperature":
                tool_outputs.append({"tool_call_id": tool.id, "output": "57"})
            elif tool.function.name == "get_rain_probability":
                tool_outputs.append({"tool_call_id": tool.id, "output": "0.30"})
        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs, run_id)
        
    def submit_tool_outputs(self, tool_outputs, run_id):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
        
    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        # Append the delta value (partial text) to the results list
        self.results.append(delta.value)
        # Call the method to update the Jupyter Notebook cell
        self.update_output()

    def update_output(self):
        """Update the Jupyter Notebook cell with the current markdown content."""
        # Clear the current output in the Jupyter Notebook cell
        clear_output(wait=True)
        # Join all the text fragments stored in results to form the complete markdown content
        markdown_content = "".join(self.results)
        # Display the markdown content in the Jupyter Notebook cell
        display(Markdown(markdown_content))

We also have to create an Assistant that knows to call our functions. Note the tools section to facilitate function calling with the proper names.

In [ ]:

assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-4o",
    tools=[
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Get the current temperature for a specific location",
            "parameters": {
            "type": "object",
            "properties": {
                "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                "type": "string",
                "enum": ["Celsius", "Fahrenheit"],
                "description": "The temperature unit to use. Infer this from the user's location."
                }
            },
            "required": ["location", "unit"]
            }
        }
        },
        {
        "type": "function",
        "function": {
            "name": "get_rain_probability",
            "description": "Get the probability of rain for a specific location",
            "parameters": {
            "type": "object",
            "properties": {
                "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
                }
            },
            "required": ["location"]
            }
        }
        }
    ]
)


Finally, let's set up a thread with our message and stream it. 

In [ ]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather in San Francisco today and the likelihood it will rain? Give me the results in a little table",
)

In [ ]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler()
    ) as stream:
    stream.until_done()

## Advanced Function Calling with a 3rd Party API

Now that we know we can do simple function calling, let's do some advanced calling to a 3rd party API to get some data back. 

In [ ]:
import requests
import json
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
from IPython.display import clear_output, display, Markdown

In [ ]:
# Create an OpenAI client instance
client = OpenAI()

We will call the National Weather Service API that gives us hourly updates. In our case, we are interested in the hourly temperature updates.

Interestingly, the NWS API requires you pass in the latitude and longitude of the city you are interested in so we wind up calling another API (geocode) to get the lat/long THEN we call the NWS API to get the temperature.

In [ ]:
import requests

# Calling the National Weather Service API to get the latest temperature
# https://weather-gov.github.io/api/general-faqs

def get_lat_lon(city_name, state_name=None):
    """
    Get the latitude and longitude for a given city (and state).
    Uses the geocode.xyz API to get the coordinates.
    Parameters:
        city_name (str): The name of the city
        state_name (str, optional): The name of the state (if applicable)
    Returns:
        tuple: Latitude and longitude as floats, or (None, None) if not found
    """
    if state_name:
        location = f"{city_name},{state_name}"
    else:
        location = city_name
    
    # URL for the geocode API
    geocode_url = f"https://geocode.xyz/{location}?json=1"
    response = requests.get(geocode_url)
    
    if response.status_code == 200:
        data = response.json()
        if 'latt' in data and 'longt' in data:
            # Return the latitude and longitude
            return float(data['latt']), float(data['longt'])
        else:
            # Error handling if coordinates are not found
            print("Error: Could not find latitude and longitude for the location.")
            return None, None
    else:
        # Error handling for failed request
        print("Error: Geocoding request failed.")
        return None, None

def get_weather_forecast(lat, lon):
    """
    Get the weather forecast for a given latitude and longitude.
    Uses the weather.gov API to get the forecast.
    Parameters:
        lat (float): Latitude
        lon (float): Longitude
    Returns:
        int: Temperature in Fahrenheit, or None if failed
    """
    # URL to get weather points based on coordinates
    points_url = f"https://api.weather.gov/points/{lat},{lon}"
    response = requests.get(points_url, headers={'User-Agent': 'MyWeatherApp (contact@example.com)'})
    
    if response.status_code == 200:
        data = response.json()
        forecast_hourly_url = data['properties']['forecastHourly']
        
        # Fetch the hourly forecast data
        forecast_response = requests.get(forecast_hourly_url, headers={'User-Agent': 'MyWeatherApp (contact@example.com)'})
        if forecast_response.status_code == 200:
            forecast_data = forecast_response.json()
            # Return the temperature from the first forecast period
            return forecast_data['properties']['periods'][0]['temperature']
        else:
            # Error handling for failed forecast request
            print("Error: Hourly forecast request failed.")
            return None
    else:
        # Error handling for failed points request
        print("Error: Points request failed.")
        return None

def get_temperature(city_name, state_name=None):
    """
    Get the temperature for a given city (and state).
    First gets the coordinates, then fetches the weather forecast.
    Parameters:
        city_name (str): The name of the city
        state_name (str, optional): The name of the state (if applicable)
    Returns:
        int: Temperature in Fahrenheit, or None if failed
    """
    # Get latitude and longitude for the city
    lat, lon = get_lat_lon(city_name, state_name)
    if lat is not None and lon is not None:
        # Get the weather forecast based on coordinates
        temperature = get_weather_forecast(lat, lon)
        return temperature
    else:
        return None

# Example usage of get_temperature function
city_name = "Houston"
state_name = "TX"
temperature = get_temperature(city_name, state_name)

if temperature is not None:
    print(f"The temperature in {city_name}, {state_name} is {temperature}°F")
else:
    print(f"Could not get the temperature for {city_name}, {state_name}")


Update our event handler to deal with just getting the temperature. Note the handle_requires_action changes.

In [ ]:
class EventHandler(AssistantEventHandler):
    """
    Custom event handler for handling actions required by the assistant.
    """
    def __init__(self):
        super().__init__()
        self.results = []  # Initialize an empty list to store the results

    @override
    def on_event(self, event):
        """
        Event handler for assistant events.
        Parameters:
            event (Event): The event to handle
        """
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id
            self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
        """
        Handles actions required by the assistant, specifically calling the get_temperature function.
        Parameters:
            data (dict): The data associated with the required action
            run_id (str): The ID of the current run
        """
        tool_outputs = []
        
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "get_temperature":
                # Parse the arguments for the function call
                arguments = json.loads(tool.function.arguments)
                city_name = arguments.get('city_name')
                state_name = arguments.get('state_name')
                # Call the get_temperature function and prepare the output
                temperature = get_temperature(city_name, state_name)
                tool_outputs.append({"tool_call_id": tool.id, "output": str(temperature)})
        
        self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
        """
        Submits the outputs of the tools called by the assistant.
        Parameters:
            tool_outputs (list): List of tool outputs to submit
            run_id (str): The ID of the current run
        """
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """
        Handle the event when there is a text delta (partial text).
        """
        # Append the delta value (partial text) to the results list
        self.results.append(delta.value)
        # Call the method to update the Jupyter Notebook cell
        self.update_output()

    def update_output(self):
        """
        Update the Jupyter Notebook cell with the current markdown content.
        """
        # Clear the current output in the Jupyter Notebook cell
        clear_output(wait=True)
        # Join all the text fragments stored in results to form the complete markdown content
        markdown_content = "".join(self.results)
        # Display the markdown content in the Jupyter Notebook cell
        display(Markdown(markdown_content))


Likewise, we need to modify our Assistant code to just call the get_temperature function. 

In [ ]:
# Define the assistant with instructions and available tools
current_temperature_assistant = client.beta.assistants.create(
    instructions="You are a weather bot. To get the temperature, use the get_temperature function.",
    model="gpt-4o",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_temperature",
                "description": "Get the current temperature for a specific location.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city_name": {
                            "type": "string",
                            "description": "The city (e.g. San Francisco)"
                        },
                        "state_name": {
                            "type": "string",
                            "description": "The state related to the city (e.g. CA)"
                        },
                    },
                    "required": ["city_name"]
                }
            }
        }
    ]
)


Finally, we create a thread and message then stream the results.

In [ ]:
# Create a new thread for the conversation
current_temperature_thread = client.beta.threads.create()

# Add a user message to the thread
current_temperature_message = client.beta.threads.messages.create(
    thread_id=current_temperature_thread.id,
    role="user",
    content="What's the weather in Houston,TX today? Give me the result in a little table.",
)

In [ ]:
# Stream the assistant's response to completion
with client.beta.threads.runs.stream(
    thread_id=current_temperature_thread.id,
    assistant_id=current_temperature_assistant.id,
    event_handler=EventHandler()
) as stream:
    stream.until_done()